Imports & Setup

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_openml

import warnings
warnings.filterwarnings("ignore")


Configuration: Datasets + Settings

In [8]:
DATASETS = ["adult", "credit-g"]   # both datasets
TEST_SIZE = 0.2
N_REPEATS = 20                     # required for hypothesis testing


Dataset Loader Function

In [9]:
def load_openml_dataset(name):
    dataset = fetch_openml(name=name, version=1, as_frame=True)
    X = dataset.data
    y = pd.factorize(dataset.target)[0]  # convert labels to numeric
    return X, y


MLP Baseline Builder (Simple & Weak by Design)

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def build_mlp_baseline(X, random_state=42):
    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=["object", "category"]).columns
    numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns

    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
            ("num", StandardScaler(), numeric_cols)
        ]
    )
    
    model = Pipeline([
        ("preprocessor", preprocessor),
        ("clf", MLPClassifier(
            hidden_layer_sizes=(50,),
            activation="relu",
            solver="adam",
            learning_rate_init=0.001,
            max_iter=200,
            alpha=0.0001,
            batch_size="auto",
            random_state=random_state
        ))
    ])

    return model


Run Baseline on Both Datasets

In [11]:
all_results = []

for dataset_name in DATASETS:
    print(f"\n==============================")
    print(f" Running MLP Baseline on: {dataset_name}")
    print(f"==============================\n")

    X, y = load_openml_dataset(dataset_name)

    results = []

    for seed in range(N_REPEATS):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=TEST_SIZE, random_state=seed, stratify=y
        )

        model = build_mlp_baseline(X, random_state=seed)
        model.fit(X_train, y_train)
        
        preds = model.predict(X_test)
        f1 = f1_score(y_test, preds, average="macro")

        results.append({
            "dataset": dataset_name,
            "seed": seed,
            "f1_macro": f1
        })

        print(f"{dataset_name} | Run {seed+1}/{N_REPEATS} → F1 = {f1:.4f}")

    # convert to dataframe
    df = pd.DataFrame(results)
    all_results.append(df)

    # summary for dataset
    print(f"\nSummary for {dataset_name}:")
    print("Mean F1:", df["f1_macro"].mean())
    print("Std F1:", df["f1_macro"].std())

    # save file
    output_path = f"../results/baselines/mlp_baseline_{dataset_name}.csv"
    df.to_csv(output_path, index=False)
    print("Saved results to:", output_path)



 Running MLP Baseline on: adult

adult | Run 1/20 → F1 = 0.7933
adult | Run 2/20 → F1 = 0.7878
adult | Run 3/20 → F1 = 0.7862
adult | Run 4/20 → F1 = 0.7864
adult | Run 5/20 → F1 = 0.7802
adult | Run 6/20 → F1 = 0.7769
adult | Run 7/20 → F1 = 0.7661
adult | Run 8/20 → F1 = 0.7726
adult | Run 9/20 → F1 = 0.7795
adult | Run 10/20 → F1 = 0.7715
adult | Run 11/20 → F1 = 0.7837
adult | Run 12/20 → F1 = 0.7784
adult | Run 13/20 → F1 = 0.7754
adult | Run 14/20 → F1 = 0.7875
adult | Run 15/20 → F1 = 0.7722
adult | Run 16/20 → F1 = 0.7821
adult | Run 17/20 → F1 = 0.7842
adult | Run 18/20 → F1 = 0.7789
adult | Run 19/20 → F1 = 0.7793
adult | Run 20/20 → F1 = 0.7904

Summary for adult:
Mean F1: 0.7806338993689588
Std F1: 0.006976482591905598
Saved results to: ../results/baselines/mlp_baseline_adult.csv

 Running MLP Baseline on: credit-g

credit-g | Run 1/20 → F1 = 0.6406
credit-g | Run 2/20 → F1 = 0.7382
credit-g | Run 3/20 → F1 = 0.6696
credit-g | Run 4/20 → F1 = 0.7024
credit-g | Run 5/20 → F